In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import time


import seaborn as sns
sns.set_style("whitegrid")

from scipy.spatial.distance import jensenshannon

import urllib3
import json

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Get data from nomis

In [9]:
boroughs = {
    'Camden': 1946157246,
    'City of London': 1946157247,
    'Hackney': 1946157248,
    'Haringey': 1946157250,
    'Islington': 1946157251,
    'Kensington and Chelsea': 1946157252,
    'Lambeth': 1946157253,
    'Lewisham': 1946157254,
    'Newham': 1946157255,
    'Southwark': 1946157256,
    'Tower Hamlets': 1946157257,
    'Wandsworth': 1946157258,
    'Westminster': 1946157259,
    'Barking and Dagenham': 1946157260,
    'Barnet': 1946157261,
    'Bexley': 1946157262,
    'Brent': 1946157263,
    'Bromley': 1946157264,
    'Croydon': 1946157265,
    'Ealing': 1946157266,
    'Enfield': 1946157267,
    'Greenwich': 1946157268,
    'Harrow': 1946157269,
    'Havering': 1946157270,
    'Hillingdon': 1946157271,
    'Hounslow': 1946157272,
    'Kingston upon Thames': 1946157273,
    'Merton': 1946157274,
    'Redbridge': 1946157275,
    'Richmond upon Thames': 1946157276,
    'Sutton': 1946157277,
    'Waltham Forest': 1946157278,
    'Hammersmith and Fulham': 1946157249,
}

In [10]:
http = urllib3.PoolManager()

In [21]:
hh = {}

for b in tqdm(boroughs):

    res = http.request("GET", "https://www.nomisweb.co.uk/api/v01/dataset/NM_538_1.data.csv?geography={}&rows=cell&cols=rural_urban&measures=20100".format(boroughs[b]))

    with open('tmp.txt', 'w') as f:
        f.write(res.data.decode('utf-8'))

    df = pd.read_csv('tmp.txt')

    vals = df['GROUP 1: OBS_VALUE'].values

    hh[b] = vals

    time.sleep(1)

  0%|          | 0/33 [00:00<?, ?it/s]D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  
  3%|▎         | 1/33 [00:01<00:36,  1.13s/it]D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  
  6%|▌         | 2/33 [00:02<00:34,  1.13s/it]D:\Users\glede\anaconda3\envs\dev\lib\site-packages\urllib3\connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  
  9%|▉         | 3/33 [0

In [22]:
idx = ['Total', '1', '2', '3', '4', '5', '6', '7', '8+']

In [23]:
df = pd.DataFrame(hh, index=idx)

In [25]:
df.to_csv('../../data/testing/nomis/hh_size.csv')

# Compute values for synthetic data

In [2]:
boroughs = ['Camden', 'City of London', 'Hackney', 'Haringey', 'Islington', 'Kensington and Chelsea', 'Lambeth', 'Lewisham', 'Newham', 'Southwark', 'Tower Hamlets', 'Wandsworth', 'Westminster', 'Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Harrow', 'Havering', 'Hillingdon', 'Hounslow', 'Kingston upon Thames', 'Merton', 'Redbridge', 'Richmond upon Thames', 'Sutton', 'Waltham Forest', 'Hammersmith and Fulham']
idx = ['Total', '1', '2', '3', '4', '5', '6', '7', '8+']

In [9]:
# ciDATGAN
dct = {}

for b in boroughs:
    df = pd.read_csv('../../data/ciDATGAN/{}.csv'.format(b))

    vals = []
    vals.append(np.sum(1/df['hh_people']))
    for i in range(1, 8):
        vals.append(np.sum(1/df[df['hh_people'] == i]['hh_people']))
    vals.append(np.sum(1/df[df['hh_people'] >= 8]['hh_people']))

    dct[b] = vals

In [10]:
df = pd.DataFrame(dct, index=idx)
df.to_csv('../../data/testing/ciDATGAN/hh_size.csv')

In [11]:
# DATGAN
dct = {}

for b in boroughs:
    df = pd.read_csv('../../data/DATGAN/{}.csv'.format(b))

    vals = []
    vals.append(np.sum(1/df['hh_people']))
    for i in range(1, 8):
        vals.append(np.sum(1/df[df['hh_people'] == i]['hh_people']))
    vals.append(np.sum(1/df[df['hh_people'] >= 8]['hh_people']))

    dct[b] = vals

In [12]:
df = pd.DataFrame(dct, index=idx)
df.to_csv('../../data/testing/DATGAN/hh_size.csv')

In [42]:
# oversample
dct = {}

for b in boroughs:
    df = pd.read_csv('../../data/oversample/{}.csv'.format(b))

    vals = []
    vals.append(np.sum(1/df['hh_people']))
    for i in range(1, 8):
        vals.append(np.sum(1/df[df['hh_people'] == i]['hh_people']))
    vals.append(np.sum(1/df[df['hh_people'] >= 8]['hh_people']))

    dct[b] = vals

In [43]:
df = pd.DataFrame(dct, index=idx)
df.to_csv('../../data/testing/oversample/hh_size.csv')

# Compare

In [2]:
orig = pd.read_csv('../../data/testing/nomis/hh_size.csv', index_col=0)

In [3]:
files = [
    '../../data/testing/ciDATGAN/hh_size.csv',
    '../../data/testing/DATGAN/hh_size.csv',
    '../../data/testing/oversample/hh_size.csv'
]

dfs = [pd.read_csv(f, index_col=0) for f in files]
labels = ['ciDATGAN', 'DATGAN', 'Oversample LTDS']

In [4]:
orig = orig/orig.iloc[0]
orig = orig.iloc[1:-1]

errors = {}

for (df, l) in zip(dfs, labels):
    df = df/df.iloc[0]
    df = df.iloc[1:-1]

    errors[l] = []

    for c in orig.columns:
        errors[l].append(jensenshannon(df[c], orig[c]))

In [5]:
for l in labels:
    print('{}: {:.2e} \pm {:.2e}'.format(l, np.mean(errors[l]), np.std(errors[l])))

ciDATGAN: 5.94e-02 \pm 3.17e-02
DATGAN: 6.36e-02 \pm 3.28e-02
Oversample LTDS: 5.47e-02 \pm 2.38e-02
